In [1]:
import torch
print(torch.__version__)

2.5.1+cu124


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 w

In [4]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00


In [5]:
from transformers import  AutoTokenizer, TrainingArguments, Trainer , AutoModelForQuestionAnswering
from datasets import load_dataset
import evaluate
import numpy as np

In [6]:
# 1. Download Pretrained Model and Tokenizer
model_name = "distilbert/distilbert-base-cased-distilled-squad"  # Or "bert-base-uncased" for TinyBERT, but DistilBERT for QA
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [7]:
# 2. Download ScienceQA Dataset (Text Only)
scienceqa_train = load_dataset("derek-thomas/ScienceQA", "default", split="train", streaming=True)
scienceqa_validation = load_dataset("derek-thomas/ScienceQA", "default", split="validation", streaming=True)

README.md:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

In [8]:
# from transformers import DistilBertModel
# model_name = "distilbert-base-uncased"
# model = DistilBertModel.from_pretrained(model_name, torch_dtype=torch.float16, attn_implementation="sdpa")

In [9]:
# 3. Data Preprocessing for Question Answering (Modified for Streaming)
def preprocess_function(examples):  # No longer batched
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = []
    for i, offsets in enumerate(offset_mapping):
        answer = examples["answer"][i]
        start_char = answer["text"][0]
        end_char = answer["text"][-1]
        start_token_idx = None
        end_token_idx = None
        for idx, (start, end) in enumerate(offsets):
            if start <= start_char <= end:
                start_token_idx = idx
            if start <= end_char <= end:
                end_token_idx = idx
        if start_token_idx is None or end_token_idx is None:
            answers.append({'start_positions': 0, 'end_positions': 0})
            continue
        answers.append({
            'start_positions': start_token_idx,
            'end_positions': end_token_idx,
        })
    inputs.update(answers)
    return inputs

In [10]:
import accelerate

accelerate.__version__

'1.3.0'

In [16]:
# Training Arguments
training_args = TrainingArguments(
    output_dir="./scienceqa-qa",  # Output directory
    evaluation_strategy="epoch",  # Evaluation strategy
    per_device_train_batch_size=8,  # Batch size (adjust based on resources)
    per_device_eval_batch_size=8,
    num_train_epochs=3,  # Number of training epochs
    learning_rate=5e-5,  # Learning rate
    weight_decay=0.01,  # Weight decay
    warmup_steps=500,  # Warmup steps
    fp16=True,  # Use mixed precision training if GPU supports it
    push_to_hub=False, # Set to True to push to hub
    max_steps=5
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
# 4. Fine-tuning and Evaluation Metrics
metric = evaluate.load("squad")  # Use the SQuAD metric

def compute_metrics(p):
    start_logits, end_logits = p.predictions
    start_positions = p.label_ids[0]
    end_positions = p.label_ids[1]
    start_pred = np.argmax(start_logits, axis=-1)
    end_pred = np.argmax(end_logits, axis=-1)

    # Need to convert to a format that metric expects
    formatted_predictions = []
    formatted_references = []

    for i in range(len(start_positions)):
        prediction = {'prediction_text': tokenizer.decode(start_pred[i], end_pred[i]), 'id': str(i)}
        reference = {'answers': [{'text': tokenizer.decode(start_positions[i], end_positions[i]), 'answer_start': 0}], 'id': str(i)}
        formatted_predictions.append(prediction)
        formatted_references.append(reference)

    return metric.compute(predictions=formatted_predictions, references=formatted_references)


In [18]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=scienceqa_train.map(
        preprocess_function, batched=True, remove_columns=scienceqa_train.column_names, batch_size=10 # Add batch size here
    ),  # <--- MAP CALL ADDED HERE (TRAIN)
    eval_dataset=scienceqa_validation.map(
        preprocess_function, batched=True, remove_columns=scienceqa_validation.column_names, batch_size=10 # Add batch size here
    ),  # <--- MAP CALL ADDED HERE (VALIDATION)
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-18-f09c08107396>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
# 4. Fine-tuning and Evaluation Metrics
metric = evaluate.load("squad")  # Use the SQuAD metric

In [20]:
def compute_metrics(p):
    start_logits, end_logits = p.predictions
    start_positions = p.label_ids[0]
    end_positions = p.label_ids[1]
    start_pred = np.argmax(start_logits, axis=-1)
    end_pred = np.argmax(end_logits, axis=-1)

    # Need to convert to a format that metric expects
    formatted_predictions = []
    formatted_references = []

    for i in range(len(start_positions)):
        prediction = {'prediction_text': tokenizer.decode(start_pred[i], end_pred[i]), 'id': str(i)}
        reference = {'answers': [{'text': tokenizer.decode(start_positions[i], end_positions[i]), 'answer_start': 0}], 'id': str(i)}
        formatted_predictions.append(prediction)
        formatted_references.append(reference)

    return metric.compute(predictions=formatted_predictions, references=formatted_references)

In [21]:
# Fine-tune the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:


Abort: 

In [ ]:
# Save the fine-tuned model
trainer.save_model("./scienceqa-qa-fine-tuned") #save model